<a href="https://colab.research.google.com/github/Filarh/Hello-World/blob/master/DeFooocusfinal_fixed_folder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🎨 DeFooocus Setup
# @markdown ### Configuración básica

# @markdown ### Selección de versión
version = "Fooocus"  # @param ["DeFooocus", "Fooocus"]

activar_drive = True  # @param {type:"boolean"}
modificar_modelo = True  # @param {type:"boolean"}

# @markdown ### Configuración del modelo
# Se utiliza un parámetro desplegable para elegir el preset deseado.
modelo_seleccion = "ZavyChromaXL_v10 (realismo, fotos)"  # @param ["ZavyChromaXL_v10 (realismo, fotos)", "SDVN8-ArtXL (Arte y vectores)", "Riot Diffusion (Generación estilo Riot)", "Custom"]
custom_link = ""  # @param {type:"string"}  # Se utiliza solo si se elige "Custom"

aspect_ratio = "896*1216"  # @param {type:"string"}
formato = "safetensors"  # @param {type:"string"}

# @markdown ### Configuración de tokens (opcional)
tokenCIVITAI = ""  # @param {type:"string"}
tokenHUGGINGFACE = ""  # @param {type:"string"}

# @markdown ### Configuración de la interfaz
theme = "dark" # @param ["dark", "light"]
preset = "default"  # @param ["default", "anime", "dpo", "hypersd", "lcm", "lightning", "playground_v2.5", "realistic", "sai", "sd1.5", "spo", "turbo"]
advenced_args = "--share --attention-split --always-high-vram --disable-offload-from-vram --all-in-fp16"  # @param {type:"string"}

import re
import os
import json
import sys
from google.colab import drive

# Function to print in color (Colab compatible)
def print_colored(text, color):
    color_codes = {
        "red": "\033[91m",
        "green": "\033[92m",
        "yellow": "\033[93m",
        "blue": "\033[94m",
        "magenta": "\033[95m",
        "cyan": "\033[96m",
        "white": "\033[97m",
        "reset": "\033[0m"
    }
    print(f"{color_codes.get(color, color_codes['reset'])}{text}{color_codes['reset']}")

# Determinar el enlace y nombre del modelo en función de la opción seleccionada
def get_model_info(modelo_seleccion, custom_link):
    modelo_presets = {
        "ZavyChromaXL_v10 (realismo, fotos)": {
            "link": "https://civitai.com/api/download/models/916744?type=Model&format=SafeTensor&size=full&fp=fp16",
            "nombre": "ZavyChromaXL_v10"
        },
        "SDVN8-ArtXL (Arte y vectores)": {
            "link": "https://civitai.com/api/download/models/212479?type=Model&format=SafeTensor&size=full&fp=fp16",
            "nombre": "SDVN8-ArtXL"
        },
        "Riot Diffusion (Generación estilo Riot)": {
            "link": "https://civitai.com/api/download/models/181589?type=Model&format=SafeTensor&size=pruned&fp=bf16",
            "nombre": "RiotDiffusion"
        },
        "Custom": {
            "link": custom_link.strip(),
            "nombre": "",  # Se establecerá después de la descarga
            "is_custom": True
        }
    }

    selected = modelo_presets.get(modelo_seleccion, None)

    if selected is None:
        print_colored(f"Error: Modelo '{modelo_seleccion}' no reconocido", "red")
        return None, None, False

    if modelo_seleccion == "Custom":
        if not selected["link"]:
            print_colored("Error: Para el modelo Custom, por favor proporciona un enlace en 'custom_link'.", "red")
            return None, None, False
        return selected["link"], selected["nombre"], True

    return selected["link"], selected["nombre"], False

# Function to modify JSON with provided parameters
def modificar_json(nombre_archivo, link_archivo, aspect_ratio, formato):
    try:
        carpeta = '/content/DeFooocus/presets' if version == "DeFooocus" else '/content/Fooocus/presets'
        if not os.path.exists(carpeta):
            print_colored(f"Error: Carpeta de presets no encontrada: {carpeta}", "red")
            return None

        archivos = os.listdir(carpeta)

        json_encontrado = None
        if 'default.json' in archivos:
            json_encontrado = 'default.json'
            print_colored("Using default.json as a reference", "green")
        else:
            json_encontrado = next((archivo for archivo in archivos if archivo.endswith('.json')), None)
            print_colored("No default.json found, using a random JSON file.", "yellow")

        if not json_encontrado:
            print_colored("No JSON files found in the specified folder.", "red")
            return None

        with open(os.path.join(carpeta, json_encontrado)) as file:
            data = json.load(file)

        archivo_modelo = f"{nombre_archivo}.{formato}"
        data['default_model'] = archivo_modelo
        data['default_aspect_ratio'] = aspect_ratio
        data['default_refiner'] = ""

        data['checkpoint_downloads'] = {archivo_modelo: link_archivo}

        nuevo_nombre = f"{nombre_archivo}.json"
        with open(os.path.join(carpeta, nuevo_nombre), 'w') as file:
            json.dump(data, file, separators=(',', ':'))

        print_colored(f"Created {nuevo_nombre} with the provided parameters!", "green")
        return nombre_archivo  # Return preset name for args
    except Exception as e:
        print_colored(f"Error al modificar JSON: {str(e)}", "red")
        return None

# Function to download the model using aria2 if it does not exist.
def download_model_aria2(url, nombre_archivo, formato, tokenCIVITAI, tokenHUGGINGFACE, is_custom=False):
    try:
        model_dir = "/content/DeFooocus/models/checkpoints" if version == "DeFooocus" else "/content/Fooocus/models/checkpoints"
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        # Append token if needed
        if "huggingface" in url and tokenHUGGINGFACE:
            url += f"&token={tokenHUGGINGFACE}"
        elif "civitai" in url and tokenCIVITAI:
            url += f"&token={tokenCIVITAI}"

        if not is_custom:
            archivo_modelo = f"{nombre_archivo}.{formato}"
            destino = os.path.join(model_dir, archivo_modelo)

            if os.path.exists(destino):
                print_colored(f"File {archivo_modelo} already exists. No download required.", "cyan")
                return destino, archivo_modelo

            download_command = f'aria2c -x 16 -s 16 -k 1M "{url}" -d "{model_dir}" -o "{archivo_modelo}"'
        else:
            # Para descarga custom: no se especifica nombre (flag -o) y se obtiene el nombre descargado
            existing_files = set(os.listdir(model_dir))
            download_command = f'aria2c -x 16 -s 16 -k 1M "{url}" -d "{model_dir}"'

        print_colored(f"Downloading from URL: {url}", "blue")
        os.system(download_command)

        if is_custom:
            new_files = set(os.listdir(model_dir)) - existing_files
            new_files = [f for f in new_files if f.endswith(f".{formato}")]
            if len(new_files) == 0:
                print_colored("Failed to download file. Verifica si añadiste la clave API correctamente.", "red")
                return None, None
            elif len(new_files) == 1:
                archivo_modelo = new_files[0]
            else:
                # Si se descargaron varios, se elige el más reciente
                archivo_modelo = sorted(new_files, key=lambda f: os.path.getmtime(os.path.join(model_dir, f)), reverse=True)[0]
            destino = os.path.join(model_dir, archivo_modelo)
        else:
            if not os.path.exists(destino):
                print_colored("Failed to download file. Verifica si añadiste la clave API correctamente.", "red")
                return None, None

        print_colored(f"Downloaded file at: {destino}", "green")
        return destino, archivo_modelo

    except Exception as e:
        print_colored(f"Download error: {str(e)}", "red")
        return None, None

# Function to setup Google Drive integration
def setup_drive_integration():
    try:
        drive.mount('/content/drive')
        nuevo_dir_lora = '/content/drive/MyDrive/MyLoras'
        nuevo_dir_salida = '/content/drive/MyDrive/Fooocus_output'

        # Create output directories if they don't exist
        os.makedirs(nuevo_dir_lora, exist_ok=True)
        os.makedirs(nuevo_dir_salida, exist_ok=True)

        launch_py_path = '/content/DeFooocus/launch.py' if version == "DeFooocus" else '/content/Fooocus/launch.py'

        if not os.path.exists(launch_py_path):
            print_colored("launch.py file not found.", "red")
            return False

        with open(launch_py_path, 'r') as file:
            launch_py_content = file.read()

        codigo_a_insertar = f'''
import os
import json

nuevo_dir_lora = '{nuevo_dir_lora}'
nuevo_dir_salida = '{nuevo_dir_salida}'

config_path = os.path.abspath("./config.txt")
config_dict = {{}}

if os.path.exists(config_path):
    with open(config_path, "r", encoding="utf-8") as json_file:
        config_dict = json.load(json_file)

config_dict['path_loras'] = os.path.abspath(nuevo_dir_lora)
config_dict['path_outputs'] = os.path.abspath(nuevo_dir_salida)

with open(config_path, "w", encoding="utf-8") as json_file:
    json.dump(config_dict, json_file, indent=4)
'''

        if codigo_a_insertar not in launch_py_content:
            with open(launch_py_path, 'w') as file:
                file.write(codigo_a_insertar + launch_py_content)
            print_colored("Drive integration configured successfully.", "green")
        else:
            print_colored("Drive integration already configured.", "cyan")

        return True
    except Exception as e:
        print_colored(f"Error setting up Drive integration: {str(e)}", "red")
        return False

# Function to setup environment and repositories
def setup_environment():
    try:
        # Check if already installed
        if os.path.exists('/content/DeFooocus') or os.path.exists('/content/Fooocus'):
            print_colored(f"Repository for {version} already exists.", "cyan")
            return True

        # Install dependencies
        print_colored("Installing necessary dependencies...", "blue")
        os.system('apt install -y aria2')
        os.system('pip install pygit2==1.12.2')

        # Fix: Create the 'cached_download' folder in the 'huggingface_hub' path if it doesn't exist
        huggingface_hub_path = '/usr/local/lib/python3.10/dist-packages/huggingface_hub'
        cached_download_path = os.path.join(huggingface_hub_path, "cached_download")
        if not os.path.exists(cached_download_path):
            os.makedirs(cached_download_path)

        # Clone repository
        os.chdir('/content')
        if version == "DeFooocus":
            print_colored("Cloning DeFooocus repository...", "blue")
            os.system('git clone https://github.com/ehristoforu/DeFooocus.git')
            os.chdir('/content/DeFooocus')
        else:
            print_colored("Cloning Fooocus repository...", "blue")
            os.system('git clone https://github.com/lllyasviel/Fooocus.git')
            os.chdir('/content/Fooocus')

        print_colored(f"{version} repository set up successfully.", "green")
        return True
    except Exception as e:
        print_colored(f"Error setting up environment: {str(e)}", "red")
        return False

# Main function to orchestrate the process
def main():
    try:
        # Setup environment
        if not setup_environment():
            return

        # Setup drive integration if required
        if activar_drive and not setup_drive_integration():
            print_colored("Warning: Drive integration failed, continuing without it.", "yellow")

        # Get model information
        link_archivo, nombre_archivo, is_custom = get_model_info(modelo_seleccion, custom_link)
        if link_archivo is None:
            return

        # Download model
        print_colored("Downloading model...", "blue")
        model_sd, downloaded_filename = download_model_aria2(link_archivo, nombre_archivo, formato, tokenCIVITAI, tokenHUGGINGFACE, is_custom=is_custom)

        if not model_sd:
            print_colored("Model download failed. Cannot continue.", "red")
            return

        # Set model name for custom models based on downloaded filename
        if is_custom:
            nombre_archivo = os.path.splitext(downloaded_filename)[0].replace(" ", "")
            print_colored(f"Using model name from downloaded file: {nombre_archivo}", "cyan")

        # Modify JSON if needed
        preset_arg = ""
        if modificar_modelo:
            preset_name = modificar_json(nombre_archivo, link_archivo, aspect_ratio, formato)
            if preset_name:
                preset_arg = f"--preset {preset_name}"
            else:
                print_colored("JSON modification failed. Using default preset.", "yellow")

        # Build launch arguments
        if preset != "default":
            args = f"{advenced_args} --theme {theme} --preset {preset}"
        else:
            args = f"{advenced_args} --theme {theme} {preset_arg}"

        # Launch the application
        print_colored(f"[{version}] Starting...", "magenta")
        os.system(f"python entry_with_update.py {args}")

    except Exception as e:
        print_colored(f"Error en la ejecución principal: {str(e)}", "red")
        import traceback
        print_colored(traceback.format_exc(), "red")

# Execute main function
if __name__ == "__main__" or True:  # Always execute in Colab
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Code already inserted in launch.py
Downloaded file at: /content/Fooocus/models/checkpoints/riotDiffusionXLLeagueOfLegendsSplash_v20.safetensors
Model downloaded: /content/Fooocus/models/checkpoints/riotDiffusionXLLeagueOfLegendsSplash_v20.safetensors
Using default.json as a reference
Created riotDiffusionXLLeagueOfLegendsSplash_v20.json with the provided parameters and structure!
[DeFooocus] Preparing...
[DeFooocus] Starting...
Already up-to-date
Update succeeded.
[System ARGV] ['entry_with_update.py', '--share', '--attention-split', '--always-high-vram', '--disable-offload-from-vram', '--all-in-fp16', '--theme', 'dark', '--preset', 'riotDiffusionXLLeagueOfLegendsSplash_v20']
Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Fooocus version: 2.5.5
Loaded preset: /content/Fooocus/presets/riotDiffusionXLLeagueOfLegendsSplash_v20.json
Failed to load conf